<font size='4'><b>Notebook content:</b> Loads the pretrained keras model and converts it into a _Sagemaker Model_. The converted Tensorflow serving model will get deployed and tested.<font/>

<br/><br/>

In [1]:
# Importing libraries.
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow.serving import Model

import boto3, re
import shutil
import tarfile

import tensorflow as tf
import tensorflow import keras
from tensorflow.keras.models import model_from_json
from tensorflow.python.saved_model import builder
from tensorflow.compat.v1.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

import json
import numpy as np
import io


In [171]:
# Retaining warnings.

In [2]:
sagemaker_session = sagemaker.Session()

In [125]:
role = get_execution_role()

In [8]:
!mkdir keras_model

mkdir: cannot create directory ‘keras_model’: File exists


#### Note: Upload the model after folder creation.

In [9]:
# Checking for file's presence.
!ls keras_model

vgg19_model.json  vgg19_model-weights.h5


#### Loading the model from disk to memory.

In [5]:
json_file = open('/home/ec2-user/SageMaker/keras_model/'+\
                 'vgg19_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
#cust_objects={"GlorotUniform":tensorflow.keras.initializers.glorot_uniform()}

loaded_model = model_from_json(loaded_model_json,custom_objects={"GlorotUniform":tf.keras.initializers.glorot_uniform})


In [6]:
loaded_model.load_weights(\
    '/home/ec2-user/SageMaker/keras_model/vgg19_model-weights.h5')
print("Loaded model from disk")

Loaded model from disk


In [7]:
# This directory structure need to be followed.
model_version = '1'
export_dir = 'export/Servo/' + model_version

In [8]:
pwd


'/home/ec2-user/SageMaker'

In [9]:
! rmdir export_dir

rmdir: failed to remove ‘export_dir’: No such file or directory


In [11]:
!mkdir -p export/Servo/1

In [12]:
shutil.rmtree(export_dir)

In [13]:
# Building the Protocol Buffer SavedModel at 'export_dir'.
build = builder.SavedModelBuilder(export_dir)

In [14]:
import tensorflow as tf
if tf.executing_eagerly():
   tf.compat.v1.disable_eager_execution()

In [15]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [16]:
# Creating prediction signature to be used by TensorFlow Serving Predict API.
signature = predict_signature_def(
    inputs= {'images':loaded_model.input}, \
    outputs={"scores": loaded_model.output})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


In [17]:
import tensorflow.python.keras.backend as K

with K.get_session() as sess:
    # Save the meta graph and variables
    build.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], \
        signature_def_map={"serving_default": signature})
    build.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1/saved_model.pb


#### Converting TensorFlow model to a SageMaker readable format.

In [20]:
!ls export

Servo


In [21]:
!ls export/Servo

1


In [22]:
!ls export/Servo/1/variables

#### Tarring the entire directory and uploading to S3.

In [23]:
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [27]:
##!saved_model_cli show --all --dir {export_path}

In [139]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', \
                key_prefix='model')


In [140]:
sagemaker_session.default_bucket()

'sagemaker-us-east-2-755283570834'

In [141]:
# One can even right 'inputs' instead of specifying path.
sagemaker_model = Model(model_data = 's3://' + \
    sagemaker_session.default_bucket() + '/model/model.tar.gz',
    role = ROLE,
    framework_version = '2.3.0',
    sagemaker_session=sagemaker_session,
    entry_point = 'train.py')


Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


### Deploying.

In [142]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1, 
                                   instance_type='ml.m4.xlarge')

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------!CPU times: user 333 ms, sys: 11.7 ms, total: 344 ms
Wall time: 7min 32s


In [143]:
predictor.endpoint

'tensorflow-inference-2020-10-25-14-18-44-611'

In [159]:
endpoint_name ='tensorflow-inference-2020-10-25-14-18-44-611'

In [160]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(\
        endpoint_name, sagemaker_session)

### Testing deployment.

#### Note: This code is for testing within AWS in a notebook. To test from outside, one should authenticate and specify region as an endpoint name is unique per AWS region.

In [18]:
# see above, should be set to the current value.
endpoint_name = 'tensorflow-inference-2020-10-25-14-18-44-611' 
runtime = boto3.client('runtime.sagemaker', region_name='us-east-2') 

In [167]:
# Fetching a sample image.
img='/home/ec2-user/SageMaker/samples/parasitized/C33P1thinF_IMG_20150619_114756a_cell_179.png'

In [168]:
result = predictor.predict(data)

In [169]:
import numpy as np
result = json.loads(result)

# which category has the highest confidence?
a = np.argmax(result)

In [166]:
if(a==1):
    print("Uninfected")
else:
    print("Infected")

Infected


### Deleting the endpoint.

#### Note: Below command deletes the endpoint and cleans the specific bucket.

In [170]:
import sagemaker

sagemaker.Session().delete_endpoint('tensorflow-inference-2020-10-25-14-18-44-611')
bucket_to_delete = boto3.resource('s3').Bucket(sagemaker_session.default_bucket())
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '2R9GAK0Y4H6G7SER',
   'HostId': 'oW/41ZVfhSX1oz9KgAuk/K07ercBlYgRT/w3UcRyz+BtSoO+dHylDZwyoxINlALhW4y9u5BnSkI=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'oW/41ZVfhSX1oz9KgAuk/K07ercBlYgRT/w3UcRyz+BtSoO+dHylDZwyoxINlALhW4y9u5BnSkI=',
    'x-amz-request-id': '2R9GAK0Y4H6G7SER',
    'date': 'Sun, 25 Oct 2020 17:26:53 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'model/model.tar.gz'},
   {'Key': 'tensorflow-inference-2020-10-25-14-18-44-446/model.tar.gz'}]}]

## Notebook's end.